In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.layers import LSTM
# from keras.optimizers import RMSprop
# from keras.utils.data_utils import get_file
import numpy as np
# import matplotlib.pyplot as plt

### Read Data

In [3]:
import ptb_reader

In [4]:
train_data, valid_data, test_data, vocab = ptb_reader.ptb_raw_data("../data/")

In [5]:
words = sorted(list(set(train_data)))
V = vocab

In [6]:
# def one_hot(x, y, sentence_len):
#     imat = np.zeros((x.shape[0], sentence_len, V))
#     for j in range(x.shape[1]):
#         for i in zip(range(sentence_len), x[j]):
#             imat[j, i[0], i[1]] = 1
#     omat = np.zeros((x.shape[0], V))
#     for j in range(x.shape[0]):
#         omat[j, y[j]] = 1
#     return imat, omat

def one_hot(x, y, sentence_len):
    omat = np.zeros((x.shape[0], V))
    for j in range(x.shape[0]):
        omat[j, y[j]] = 1
    return omat

### Create model

In [7]:
tf.reset_default_graph()

In [8]:
seq_len = 20

In [9]:
# Create place holders
embedding_size = 300
with tf.variable_scope('model1'):
    data = tf.placeholder(tf.int32, [None, seq_len])
    target = tf.placeholder(tf.float32, [None, V])
    

In [10]:
with tf.variable_scope('model1'):
    # Get embedding
    embedding = tf.get_variable('embedding', [V, embedding_size])
    inputs = tf.nn.embedding_lookup(embedding, data)
    inputs = tf.nn.dropout(inputs, keep_prob = 0.5)
    # create lstm cell
    num_hidden = 128
    lstm_unit = lambda: tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([lstm_unit() for _ in range(3)], state_is_tuple=True)
    
    # pass through lstm
    val, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)

    # get lstm output
    val = tf.transpose(val, [1, 0, 2])
    last = tf.gather(val, int(val.get_shape()[0]) - 1)

    # pass lstm output to nn
    weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
    bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

    # final nn output
    predict = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [14]:
with tf.variable_scope('model1'):
    # Cross entropy loss
#     loss = -tf.reduce_sum(target*tf.log(tf.clip_by_value(predict, 1e-10, 1.0)))
    batch_size, num_steps = 100, seq_len
    weights = tf.ones([batch_size * num_steps])
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([predict], [target],[weights], V)
    optimizer = tf.train.AdamOptimizer()
    minimize = optimizer.minimize(loss)

TypeError: Value passed to parameter 'labels' has DataType float32 not in list of allowed values: int32, int64

In [12]:
with tf.variable_scope('model1'):
    # evaluations
    corrects = tf.equal(tf.argmax(predict, 1), tf.argmax(target, 1))
    accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

In [13]:
with tf.variable_scope('model1'):
    # initialization
    init_op = tf.global_variables_initializer()

### Create session and init variables

In [14]:
# create session
sess = tf.Session()

In [15]:
# init
sess.run(init_op)

### Train

In [16]:
saver = tf.train.Saver()

In [53]:
# saver.restore(sess, "saved_model/model.ckpt")

In [24]:
batch_size, num_steps = 100, seq_len # maxlen size sentence and 100 of them, num_steps: step of words
_epochs = 10
_ipochs = 4
for epoch in range(_epochs):
    print('Epoch: {}/{}'.format(epoch, _epochs))
    for step, (_x, _y) in enumerate(ptb_reader.ptb_iterator(train_data, batch_size, num_steps)):
        _y = _y[:, -1]
        y = one_hot(_x, _y, seq_len)
        for ipoch in range(_ipochs):
            sess.run(minimize, feed_dict={data: _x, target: y})
        
        if step % 20 == 0:
            acc = sess.run(accuracy, feed_dict={data: _x, target: y})
            perplexity = 2**(sess.run(loss, feed_dict={data: _x, target: y}))
            print('{}. Training Accuracy: {:0.2F} %'.format(step, float(acc)*100))
            print('{}. Training Perplexity: {}\n'.format(step, perplexity))
            save_path = saver.save(sess, "saved_model/model.ckpt")
            #print("Model saved in file: %s" % save_path)

Epoch: 0/10
0. Training Accuracy: 33.00 %
0. Training Perplexity: 3.3788477197243498e+100

20. Training Accuracy: 81.00 %
20. Training Perplexity: 1.293902553423465e+26

40. Training Accuracy: 79.00 %
40. Training Perplexity: 8.261632261424608e+26

60. Training Accuracy: 76.00 %
60. Training Perplexity: 2.2276774027109597e+29

80. Training Accuracy: 86.00 %
80. Training Perplexity: 6.717637989829769e+18

100. Training Accuracy: 81.00 %
100. Training Perplexity: 1.2938376775849196e+24

120. Training Accuracy: 80.00 %
120. Training Perplexity: 2.359612992432006e+26

140. Training Accuracy: 86.00 %
140. Training Perplexity: 1.2363047151704631e+23

160. Training Accuracy: 88.00 %
160. Training Perplexity: 1.8261116556881136e+19

180. Training Accuracy: 78.00 %
180. Training Perplexity: 1.9438461203388803e+24

200. Training Accuracy: 82.00 %
200. Training Perplexity: 6.785393024462755e+23

220. Training Accuracy: 95.00 %
220. Training Perplexity: 5.899869049018289e+18

240. Training Accurac

KeyboardInterrupt: 

### Test

In [28]:
batch_size, num_steps = 1000, seq_len # maxlen size sentence and 100 of them, num_steps: step of words
for step, (_tx, _ty) in enumerate(ptb_reader.ptb_iterator(test_data, batch_size, num_steps)):
    _ty = _ty[:, -1]
    ty = one_hot(_tx, _ty, seq_len)
    acc = sess.run(accuracy, feed_dict={data: _tx, target: ty})
    print('{}. Test Accuracy: {:0.2F} %'.format(step, float(acc)*100))

0. Test Accuracy: 7.10 %
1. Test Accuracy: 8.30 %
2. Test Accuracy: 7.90 %
3. Test Accuracy: 6.90 %


In [66]:
y_hat = sess.run(predict, feed_dict={data: _tx})
np.argmax(y_hat, axis = 1)

array([  8, 243,  16,  47,  34], dtype=int64)

In [67]:
_ty

array([ 997,    1,    2,   47, 4903])

In [40]:
sess.close()

In [69]:
_tx

array([[  60,  111,   15, 2988,    1,  442,  232,   70,   18,    1,  125,
         584,    2,    1,  558,    1,  141,    4, 2195, 5555],
       [  91,    6, 1135,  402,    7,    1, 8204,    2,   10,  862,   75,
         863,   54,    7,    1, 6986,    8,    0,   61,    7],
       [ 137,   26,  524, 5146,   11,   39,   96,    2, 1706,   90, 1968,
           4,    0, 5284,   47,   39, 1108, 9215,    0,    1],
       [1006,    0, 1422,  353,    2,   39,   93, 2102,    1, 3327,    2,
          78,   73,  314,    6,  260,   72,  195,    0,   60],
       [  93, 8014,    6,   45, 1173, 1874,    5,    0,   98,    8,    0,
         197,    1,    2, 1243,  456, 3459,   10,   38,   26]])